# Iterative Prompt Development
## Setup

In [1]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [2]:
import openai
import os

from getpass import getpass


os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


In [6]:
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [7]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [8]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing the SWC-100/SWC-110 Office Chair, a sleek and stylish addition to any home or business setting. This chair is part of a stunning family of mid-century inspired office furniture, offering a range of options to suit your preferences.

Choose from a variety of shell colors and base finishes to customize your chair to your liking. The SWC-100 model comes with plastic back and front upholstery, while the SWC-110 model offers full upholstery in a selection of 10 fabric and 6 leather options. You can also choose whether you prefer armrests or not.

Constructed with a durable 5-wheel plastic coated aluminum base, this chair features a pneumatic adjust for easy raise/lower action. The dimensions of the chair are as follows: WIDTH 53 CM, DEPTH 51 CM, HEIGHT 80 CM, SEAT HEIGHT 44 CM, and SEAT DEPTH 41 CM.

Additional options include soft or hard-floor caster choices, two seat foam densities (medium or high), and the option for armless or 8 position PU armrests. The materials used in t

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [9]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Available in various colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable construction and adjustable height feature. Made in Italy for quality assurance.


In [11]:
len(response.split())

56

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [10]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, crafted with precision and quality materials. The shell base glider is made of durable cast aluminum with a modified nylon coating, while the seat features high-density foam for ultimate comfort. Available in various colors and finishes, this chair is perfect for any home or business setting. Made in Italy.


In [12]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile office chair, part of a stylish mid-century furniture collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with a durable aluminum base and high-density foam seat for comfort. Perfect for home or office use. Product IDs: SWC-100, SWC-110.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [13]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>This mid-century inspired office chair is a perfect addition to any workspace, whether at home or in a business setting. The chair comes in a variety of shell colors and base finishes to suit your style. You can choose between plastic back and front upholstery or full upholstery in a range of fabric and leather options. The chair is designed for comfort with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. The seat is made with high-quality HD36 foam for durability and comfort. Made in Italy, this chair is not only stylish but also built to last.</p>

<p>Product IDs: SWC-100, SWC-110</p>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurements (inches)</th>
  </tr>
  <tr>
    <td>Width</td>
    <td>20.87"</td>
  </tr>
  <tr>
    <td>Depth</td>
    <td>20.08"</td>
  </tr>
  <tr>
    <td>Height</td>
    <td>31.50"</td>
  </tr>
  <tr>
    <td>Seat Height</td>
    <td>17.32"</td>
  </tr>
  <tr

## Load Python libraries to view HTML

In [14]:
from IPython.display import display, HTML

In [15]:
display(HTML(response))

Dimension,Measurements (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""
